In [2]:
import saspy
sas = saspy.SASsession(colorLOG=True)

Using SAS Config named: oda
SAS Connection established. Subprocess id is 15652



### proc freq 


In [8]:
sas.submitLST(f"""
DATA orders;
    INPUT Coffee $ Window $ @@;
    datalines;
esp w cap d cap w kon w ice w kon d esp d kon w ice d esp d
cap w esp d cap d Kon d . d kon w esp d cap w ice w kon w
kon w kon w ice d esp d kon w esp d esp w kon w cap w kon w
;
""")

df = sas.sasdata('orders').to_df()
df

,Coffee,Window
0,esp,w
1,cap,d
2,cap,w
3,kon,w
4,ice,w
5,kon,d
6,esp,d
7,kon,w
8,ice,d
9,esp,d


In [5]:
sas.submitLST(f"""
* Print tables for Window and Window by Coffee;
PROC FREQ DATA = orders; 
    TABLES Window; 
RUN;
""")

Window,Frequency,Percent,CumulativeFrequency,CumulativePercent
d,13,43.33,13,43.33
w,17,56.67,30,100.00


In [6]:
sas.submitLST(f"""
* Print tables for Window and Window by Coffee;
PROC FREQ DATA = orders; 
    TABLES Window Window * Coffee; 
RUN;
""")

### statistical options

In [14]:
sas.submitLST(f"""
* Print tables for Window and Window by Coffee;
PROC FREQ DATA = orders;
    TABLES Window * Coffee / MISSING; 
RUN;
""")

In [ ]:
sas.submitLST(f"""
* Print tables for Window and Window by Coffee;
PROC FREQ DATA = orders noprint;
    TABLES Window * Coffee / LIST MISSING out = freqout; 
RUN;
""")
df = sas.sasdata('freqout').to_df()
df

,Window,Coffee,COUNT,PERCENT
0,d,NaN,1.0,3.333333
1,d,Kon,1.0,3.333333
2,d,cap,2.0,6.666667
3,d,esp,6.0,20.000000
4,d,ice,2.0,6.666667
5,d,kon,1.0,3.333333
6,w,cap,4.0,13.333333
7,w,esp,2.0,6.666667
8,w,ice,2.0,6.666667
9,w,kon,9.0,30.000000
